# Project 2 Ames Housing



## Imports

In [4]:
# Imports:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline


# Set the graph style
sns.set_style('whitegrid')

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

## Read in data

In [5]:
# Set relative data path
data_path = "../datasets"
test_file_name = "test.csv"
train_file_name = "train.csv"

a_tst = pd.read_csv(os.path.join(data_path, test_file_name))
a_trn = pd.read_csv(os.path.join(data_path, train_file_name))


### View dataframes and data

1. Look at all columns for both the train and test data sets
2. Pick a set of features to examine first
3. Explore these features individually


In [6]:
# Look at the train and test data sets
print("Train", a_trn.shape)
print(a_trn.columns)
# print(a_trn.info())

print ("Test", a_tst.shape)
print(a_tst.columns)
# print(a_tst.info())


Train (2051, 81)
Index(['Id', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace Qu', 'Garage Type', 'Garage Y

### variable notes

* Excluded from analysis
    * Low Qual Fin SF has only 33 values that are not zero
    * 1st Flr SF + 2nd Flr SF = Gr Liv Area except for 33 values used Gr Liv Area
    * Lot Frontage has 330 missing values

    
* Missing values (NaN values)
    * Set Bsmt Full Bath NaN values to 0 (2 values)
    * Set Bsmt Half Bath NaN values to 0 (2 values)
    * Set Total Bsmt SF NaN values to 0 (1 values)
    * Set Bsmt Cond NaN values to NA (55 values in train dataset)
    * Set Garage Type NaN values to NA (113 values in train dataset)
    * Set Pool QC values to NA (2042 values in train dataset)
    * Set Misc Features values to NA (1986 values in train dataset)
        

In [26]:
# Examine Low Qual Fin SF
a_trn[a_trn['Low Qual Fin SF']!=0].shape


(33, 81)

In [130]:
# Examine '1st Flr SF' and '2nd Flr SF' and 'GR Liv Area'
a_trn[(a_trn['Gr Liv Area'] - (a_trn['1st Flr SF'] + a_trn['2nd Flr SF']))!=0][['Gr Liv Area','1st Flr SF','2nd Flr SF']]


,Gr Liv Area,1st Flr SF,2nd Flr SF
249,845,640,0
269,1200,520,600
320,1382,854,0
356,1526,1013,0
382,2229,929,929
497,1639,1166,0
705,1594,828,658
828,2377,1313,0
858,3395,1440,1440
918,1869,1188,561


In [186]:
# Examine Lot Frontage
feat_check = ['Lot Frontage','Utilities','Lot Config',
       'House Style','Year Remod/Add','Exter Qual','Exter Cond','Bsmt Cond',
       'BsmtFin Type 1','Heating QC', 'Central Air','Functional',
       'Enclosed Porch','Garage Type','Pool QC','Misc Feature', 'Mo Sold']
a = a_trn[feat_check].isnull().sum()
b = a_tst[feat_check].isnull().sum()

In [187]:
print("Null values Train: Test")
for i in range(len(a)):
    print("%s: %d, %d" % (a.index[i], a[i], b[i])) 




Null values Train: Test
Lot Frontage: 330, 160
Utilities: 0, 0
Lot Config: 0, 0
House Style: 0, 0
Year Remod/Add: 0, 0
Exter Qual: 0, 0
Exter Cond: 0, 0
Bsmt Cond: 55, 25
BsmtFin Type 1: 55, 25
Heating QC: 0, 0
Central Air: 0, 0
Functional: 0, 0
Enclosed Porch: 0, 0
Garage Type: 113, 44
Pool QC: 2042, 875
Misc Feature: 1986, 838
Mo Sold: 0, 0


In [188]:
# Create a subset list of features
targ = ['SalePrice']
feat_1 = ['MS SubClass', 'MS Zoning', 'Lot Area', 'Utilities', 'Lot Config', 'Neighborhood', 'Bldg Type',
         'House Style', 'Overall Qual', 'Overall Cond', 'Year Built', 'Year Remod/Add','Exter Qual',
         'Exter Cond', 'Bsmt Cond', 'Total Bsmt SF','Heating QC', 'Central Air',
         'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath',
         'Full Bath', 'Half Bath', 'Bedroom AbvGr', 'Kitchen Qual', 'Functional','Enclosed Porch',
         'Garage Type', 'Pool QC','Misc Feature','Mo Sold', 'Yr Sold', 'Sale Type']

X1_trn = a_trn[feat_1]
X1_tst = a_tst[feat_1]

In [208]:
# Examine the X1_trn and X1_tst
for col in X1_trn.columns:
    print("Train: %15s %5d %15s" % (col, X1_trn[col].isnull().sum(), X1_trn[col].dtypes))
    print(" Test: %15s %5d %15s" % (col, X1_tst[col].isnull().sum(), X1_tst[col].dtypes))
   

# print(X1_trn.info())
# print(X1_tst.info())


Train:     MS SubClass     0           int64
 Test:     MS SubClass     0           int64
Train:       MS Zoning     0          object
 Test:       MS Zoning     0          object
Train:        Lot Area     0           int64
 Test:        Lot Area     0           int64
Train:       Utilities     0          object
 Test:       Utilities     0          object
Train:      Lot Config     0          object
 Test:      Lot Config     0          object
Train:    Neighborhood     0          object
 Test:    Neighborhood     0          object
Train:       Bldg Type     0          object
 Test:       Bldg Type     0          object
Train:     House Style     0          object
 Test:     House Style     0          object
Train:    Overall Qual     0           int64
 Test:    Overall Qual     0           int64
Train:    Overall Cond     0           int64
 Test:    Overall Cond     0           int64
Train:      Year Built     0           int64
 Test:      Year Built     0           int64
Train:  Ye

### Handle NaN values

In [209]:
# Find the null values
col = 'Bsmt Full Bath'
df = X1_trn
df[df[col].isnull()]


,MS SubClass,MS Zoning,Lot Area,Utilities,Lot Config,Neighborhood,Bldg Type,House Style,Overall Qual,Overall Cond,...,Bedroom AbvGr,Kitchen Qual,Functional,Enclosed Porch,Garage Type,Pool QC,Misc Feature,Mo Sold,Yr Sold,Sale Type
616,20,RL,47007,AllPub,Inside,Edwards,1Fam,1Story,5,7,...,5,Ex,Typ,0,Attchd,NaN,NaN,7,2008,WD
1327,20,RM,5940,AllPub,FR3,BrkSide,1Fam,1Story,4,7,...,2,TA,Typ,0,Detchd,NaN,NaN,4,2008,ConLD


In [211]:
# Find the null values
col = 'Bsmt Half Bath'
df[df[col].isnull()]


,MS SubClass,MS Zoning,Lot Area,Utilities,Lot Config,Neighborhood,Bldg Type,House Style,Overall Qual,Overall Cond,...,Bedroom AbvGr,Kitchen Qual,Functional,Enclosed Porch,Garage Type,Pool QC,Misc Feature,Mo Sold,Yr Sold,Sale Type
616,20,RL,47007,AllPub,Inside,Edwards,1Fam,1Story,5,7,...,5,Ex,Typ,0,Attchd,NaN,NaN,7,2008,WD
1327,20,RM,5940,AllPub,FR3,BrkSide,1Fam,1Story,4,7,...,2,TA,Typ,0,Detchd,NaN,NaN,4,2008,ConLD


In [213]:
# Find the null values
col = 'Total Bsmt SF'
df[df[col].isnull()]


,MS SubClass,MS Zoning,Lot Area,Utilities,Lot Config,Neighborhood,Bldg Type,House Style,Overall Qual,Overall Cond,...,Bedroom AbvGr,Kitchen Qual,Functional,Enclosed Porch,Garage Type,Pool QC,Misc Feature,Mo Sold,Yr Sold,Sale Type
1327,20,RM,5940,AllPub,FR3,BrkSide,1Fam,1Story,4,7,...,2,TA,Typ,0,Detchd,NaN,NaN,4,2008,ConLD


In [232]:
# Find and replace null values with 0
df_l = [X1_trn, X1_tst]
col_l = ['Bsmt Full Bath','Bsmt Half Bath','Total Bsmt SF']

for df in df_l:
    for col in col_l:
        # Fill the null values with NA
        df[col] = df[col].fillna(0)


In [227]:
# Find and replace null values with NA
df_l = [X1_trn, X1_tst]
col_l = ['Bsmt Cond','Garage Type', 'Pool QC','Misc Feature']

for df in df_l:
    for col in col_l:
        # Fill the null values with NA
        df[col] = df[col].fillna("NA")


In [243]:
# Examine the X1_trn and X1_tst
for col in X1_trn.columns:
    print("Train: %15s %5d %15s" % (col, X1_trn[col].isnull().sum(), X1_trn[col].dtypes))
    print(" Test: %15s %5d %15s" % (col, X1_tst[col].isnull().sum(), X1_tst[col].dtypes))
   


Train:     MS SubClass     0           int64
 Test:     MS SubClass     0           int64
Train:       MS Zoning     0          object
 Test:       MS Zoning     0          object
Train:        Lot Area     0           int64
 Test:        Lot Area     0           int64
Train:       Utilities     0          object
 Test:       Utilities     0          object
Train:      Lot Config     0          object
 Test:      Lot Config     0          object
Train:    Neighborhood     0          object
 Test:    Neighborhood     0          object
Train:       Bldg Type     0          object
 Test:       Bldg Type     0          object
Train:     House Style     0          object
 Test:     House Style     0          object
Train:    Overall Qual     0           int64
 Test:    Overall Qual     0           int64
Train:    Overall Cond     0           int64
 Test:    Overall Cond     0           int64
Train:      Year Built     0           int64
 Test:      Year Built     0           int64
Train:  Ye

### Show maximum and minimum values

In [229]:
# Describe
X1_trn.describe().T


,count,mean,std,min,25%,50%,75%,max
MS SubClass,2051.0,57.008776,42.824223,20.0,20.0,50.0,70.0,190.0
Lot Area,2051.0,10065.208191,6742.488909,1300.0,7500.0,9430.0,11513.5,159000.0
Overall Qual,2051.0,6.112140,1.426271,1.0,5.0,6.0,7.0,10.0
Overall Cond,2051.0,5.562165,1.104497,1.0,5.0,5.0,6.0,9.0
Year Built,2051.0,1971.708922,30.177889,1872.0,1953.5,1974.0,2001.0,2010.0
Year Remod/Add,2051.0,1984.190151,21.036250,1950.0,1964.5,1993.0,2004.0,2010.0
Total Bsmt SF,2051.0,1057.471965,449.908003,0.0,793.0,994.0,1318.5,6110.0
Gr Liv Area,2051.0,1499.330083,500.447829,334.0,1129.0,1444.0,1728.5,5642.0
Bsmt Full Bath,2051.0,0.427109,0.522589,0.0,0.0,0.0,1.0,3.0
Bsmt Half Bath,2051.0,0.063384,0.251590,0.0,0.0,0.0,0.0,2.0


In [230]:
# Describe
X1_tst.describe().T


,count,mean,std,min,25%,50%,75%,max
MS SubClass,879.0,58.270762,42.211389,20.0,20.0,50.0,70.0,190.0
Lot Area,879.0,10340.920364,10047.335167,1477.0,7298.5,9453.0,11606.5,215245.0
Overall Qual,879.0,6.054608,1.374756,2.0,5.0,6.0,7.0,10.0
Overall Cond,879.0,5.565415,1.128422,1.0,5.0,5.0,6.0,9.0
Year Built,879.0,1970.533561,30.403527,1880.0,1954.0,1972.0,2000.0,2010.0
Year Remod/Add,879.0,1984.444824,20.454546,1950.0,1967.0,1992.0,2003.0,2010.0
Total Bsmt SF,879.0,1036.750853,419.258428,0.0,790.0,975.0,1248.0,3138.0
Gr Liv Area,879.0,1500.531286,517.413260,407.0,1114.0,1436.0,1781.0,4676.0
Bsmt Full Bath,879.0,0.440273,0.529983,0.0,0.0,0.0,1.0,2.0
Bsmt Half Bath,879.0,0.055745,0.229560,0.0,0.0,0.0,0.0,1.0


In [245]:
# Look at the maximum and minimum values
df = X1_trn
# check for min and max values
for column in df.columns:
    print("Column: %15s  min: %10s  max: %10s" % (column, str(df[column].min()), str(df[column].max())))
    

Column:     MS SubClass  min:         20  max:        190
Column:       MS Zoning  min:    A (agr)  max:         RM
Column:        Lot Area  min:       1300  max:     159000
Column:       Utilities  min:     AllPub  max:     NoSewr
Column:      Lot Config  min:     Corner  max:     Inside
Column:    Neighborhood  min:    Blmngtn  max:    Veenker
Column:       Bldg Type  min:       1Fam  max:     TwnhsE
Column:     House Style  min:     1.5Fin  max:       SLvl
Column:    Overall Qual  min:          1  max:         10
Column:    Overall Cond  min:          1  max:          9
Column:      Year Built  min:       1872  max:       2010
Column:  Year Remod/Add  min:       1950  max:       2010
Column:      Exter Qual  min:         Ex  max:         TA
Column:      Exter Cond  min:         Ex  max:         TA
Column:       Bsmt Cond  min:         Ex  max:         TA
Column:   Total Bsmt SF  min:        0.0  max:     6110.0
Column:      Heating QC  min:         Ex  max:         TA
Column:     Ce

In [246]:
# Look at the maximum and minimum values
df = X1_tst
# check for min and max values
for column in df.columns:
    print("Column: %15s  min: %10s  max: %10s" % (column, str(df[column].min()), str(df[column].max())))
    

Column:     MS SubClass  min:         20  max:        190
Column:       MS Zoning  min:    C (all)  max:         RM
Column:        Lot Area  min:       1477  max:     215245
Column:       Utilities  min:     AllPub  max:     NoSewr
Column:      Lot Config  min:     Corner  max:     Inside
Column:    Neighborhood  min:    Blmngtn  max:    Veenker
Column:       Bldg Type  min:       1Fam  max:     TwnhsE
Column:     House Style  min:     1.5Fin  max:       SLvl
Column:    Overall Qual  min:          2  max:         10
Column:    Overall Cond  min:          1  max:          9
Column:      Year Built  min:       1880  max:       2010
Column:  Year Remod/Add  min:       1950  max:       2010
Column:      Exter Qual  min:         Ex  max:         TA
Column:      Exter Cond  min:         Ex  max:         TA
Column:       Bsmt Cond  min:         Fa  max:         TA
Column:   Total Bsmt SF  min:          0  max:       3138
Column:      Heating QC  min:         Ex  max:         TA
Column:     Ce

In [123]:
# Show value counts
df = X1_trn
col = 'MS SubClass'
print(df[col].value_counts())

df = X1_tst
print(df[col].value_counts())


20     770
60     394
50     198
120    132
30     101
70      90
160     88
80      86
90      75
190     46
85      28
75      16
45      11
180     11
40       4
150      1
Name: MS SubClass, dtype: int64
20     309
60     181
50      89
120     60
160     41
70      38
30      38
90      34
80      32
85      20
190     15
75       7
45       7
180      6
40       2
Name: MS SubClass, dtype: int64


In [124]:
# Show value counts
df = X1_trn
col = 'MS Zoning'
print(df[col].value_counts())

df = X1_tst
print(df[col].value_counts())


RL         1598
RM          316
FV          101
C (all)      19
RH           14
A (agr)       2
I (all)       1
Name: MS Zoning, dtype: int64
RL         675
RM         146
FV          38
RH          13
C (all)      6
I (all)      1
Name: MS Zoning, dtype: int64


In [133]:
# Show value counts
df = X1_trn
col = 'Neighborhood'
print(df[col].value_counts())

df = X1_tst
print(df[col].value_counts())


NAmes      310
CollgCr    180
OldTown    163
Edwards    143
Somerst    130
NridgHt    122
Gilbert    116
Sawyer     111
NWAmes      87
SawyerW     87
Mitchel     82
BrkSide     76
Crawfor     71
IDOTRR      69
NoRidge     48
Timber      48
StoneBr     38
SWISU       32
ClearCr     27
MeadowV     24
Blmngtn     22
BrDale      19
NPkVill     17
Veenker     17
Blueste      6
Greens       3
GrnHill      2
Landmrk      1
Name: Neighborhood, dtype: int64
NAmes      133
CollgCr     87
OldTown     76
Somerst     52
Edwards     51
Gilbert     49
NridgHt     44
NWAmes      44
Sawyer      40
SawyerW     38
Crawfor     32
Mitchel     32
BrkSide     32
IDOTRR      24
Timber      24
NoRidge     23
ClearCr     17
SWISU       16
StoneBr     13
MeadowV     13
BrDale      11
Veenker      7
NPkVill      6
Blmngtn      6
Greens       5
Blueste      4
Name: Neighborhood, dtype: int64


# References

* https://towardsdatascience.com/ridge-and-lasso-regression-a-complete-guide-with-python-scikit-learn-e20e34bcbf0b
* https://stackoverflow.com/questions/41335718/keep-same-dummy-variable-in-training-and-testing-data